In [1]:
import pandas
pandas.set_option('display.max_rows',1000)
pandas.set_option('display.max_columns',10000)

In [2]:
m2016 = pandas.read_csv('music2016.csv').drop
m2016

,Unnamed: 0,artist,title,rating,url,chords
0,0,Justin Bieber,LOVE YOURSELF,4527.0,https://tabs.ultimate-guitar.com/j/justin_bieb...,C G/B Am Dm C G/B C G/B Am Dm C G/B C G/B Am D...
1,1,Justin Bieber,SORRY,737.0,https://tabs.ultimate-guitar.com/j/justin_bieb...,F Am G F Am G F Am G F Am G F Am G F Am G F Am...
2,2,Drake Featuring WizKid & Kyla,ONE DANCE,113.0,https://tabs.ultimate-guitar.com/d/drake/one_d...,Bbm C# Ebm Ebm Bbm C# Ebm Ebm Bbm C# Ebm Ebm B...
3,3,Rihanna Featuring Drake,WORK,86.0,https://tabs.ultimate-guitar.com/r/rihanna/wor...,Am7 G/B Cadd9 D Am7 G/B Cadd9 D Am7 G/B Cadd9 ...
4,4,twenty one pilots,STRESSED OUT,887.0,https://tabs.ultimate-guitar.com/t/twenty_one_...,Am C Dm E Em F G F Dm Am Am F Dm Am F Dm Am F ...
5,5,Adele,HELLO,1032.0,https://tabs.ultimate-guitar.com/a/adele/hello...,Em G D C Bm Em G D C Em G D C Em G D C Em G D ...
6,6,The Chainsmokers Featuring Daya,DON'T LET ME DOWN,284.0,https://tabs.ultimate-guitar.com/t/the_chainsm...,D A E F#m D A E F#m D A E F#m D A E F#m D A E ...
7,7,Justin Timberlake,CAN'T STOP THE FEELING!,156.0,https://tabs.ultimate-guitar.com/j/justin_timb...,C Am F Am C Am F Am C Am F Am C Am F Am C Am F...
8,8,The Chainsmokers Featuring Halsey,CLOSER,606.0,https://tabs.ultimate-guitar.com/t/the_chainsm...,Cadd9 D Em7 D Cadd9 D Em7 D Cadd9 D Em7 D Cadd...
9,9,Sia Featuring Sean Paul,CHEAP THRILLS,316.0,https://tabs.ultimate-guitar.com/s/sia/cheap_t...,F#m D A E F#m D A E F#m D A E F#m D A E F#m D ...


In [11]:
m2016['chord_set'] = [set(chords.split(' ')) for chords in m2016.chords]

In [22]:
m2016.chord_set[0].union(m2016.chord_set[1])

{'Am', 'C', 'Dm', 'F', 'G', 'G/B'}

In [23]:
m2016.chord_set[0]

{'Am', 'C', 'Dm', 'F', 'G', 'G/B'}

In [24]:
m2016.chord_set[1]

{'Am', 'Dm', 'F', 'G'}

In [26]:
vocab = m2016.chord_set[0]
for ss in m2016.chord_set:
    print(type(ss))
vocab

<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class 'set'>
<class

{'Am', 'C', 'Dm', 'F', 'G', 'G/B'}